# Fx Forward Pricing

This notebook will run through the following business use cases :
* [Pricing an Fx Forward using a supplied vendor model.](#pricing_bond)
* [Valuing a portfolio with an Fx Forward position.](#pricing_bond_portfolio)
* [Valuing our Fx Forward portfolios using different models.](#accrual_override)
* [Valuing bond PV using an externally provided market quote for the bond.](#external_bond_price)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of a bond instrument based on user provided parameters.](#bond_definition)
* [Using the StructureMarketData store to hold your OIS yield curve data in way that enables it to be discovered during the bond valuation process.](#structured_market_data)
* [Configuring recipes to run built in LUSID bond valuation models that make use of the structured data (OIS Yield Curve) you provided.](#recipe_configuration)
* [Using aggregation requests to return the accrued interest as well as the PV of the bond based on our instrument definition.](#accrued_interest)
* [Using the StructuredResultData store to override the accrued interest calculation and instead use static values.](#structured_result_data)
* [Updating our recipes to make use of the StructuredResultData entries in your valuations.](#structured_result_data)
* [Upserting bond market price as quotes and configuring recipes that value bonds by using the market quotes.](#external_bond_price)

<br>

For this notebook example we'll work the following set of GBP/USD Forwards with 2Y, 5Y and 10Y maturities:

* Trade Date : 02 Aug 2020
* Maturity Dates :
    * 02 Aug 2023 (3Y)
    * 02 Aug 2025 (5Y)
    * 02 Aug 2030 (10Y)
* Spot Rate : 1.2508
* Domestic Amount : 100,000,000
* Foreign Amounts :
    * 125,710,000 (3Y)
    * 126,485,000 (5Y)
    * 130,430,000 (10Y)


## Setup LUSID and LUSID API objects.

In [1]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from IPython.core.display import display
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)
from lusidtools.jupyter_tools.stop_execution import StopExecution


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)

# Setup the scope we'll use in this notebook:
scope = "fx-forward-pricing-nb"

LUSID Environment Initialised
LUSID SDK Version:  0.6.4740.0


In [2]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    df = pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])
    df['Effective At'] = df['Effective At'].dt.strftime('%d %b %Y')
    return df


## Define our Fx Forward Instruments

We'll start by defining our forwards using LUSID's internal representation of an Fx forward instrument. For a detailed
breakdown the internal representation take a look at the "FxForwardInstrument" section of the [LUSID API Swagger Specification](https://www.lusid.com/api/swagger/index.html)

### Initialise FxForward Parameters

Let's setup the basic parameters required for our set of Fx forwards:

In [3]:
trade_date = datetime(2020, 6, 1, tzinfo=pytz.utc)
#trade_date = datetime.today().replace(tzinfo=pytz.utc)
maturity_dates = {
    '3Y' : datetime(2023, 6, 1, tzinfo=pytz.utc),
    '5Y' : datetime(2025, 6, 1, tzinfo=pytz.utc),
    '10Y': datetime(2030, 6, 1, tzinfo=pytz.utc)
}
# maturity_dates = {
#     '3Y' : trade_date.replace(year=trade_date.year + 3),
#     '5Y' : trade_date.replace(year=trade_date.year + 5),
#     '10Y': trade_date.replace(year=trade_date.year + 6)
# }
spot_rate =  1.2508
dom_amount = 100000000
fgn_amounts = {
    '3Y' : 125710000,
    '5Y' : 126485000,
    '10Y': 130430000
}

### Create FxForwardInstrument Definitions

We now feed those parameters to construct the LUSID representation of an FxForward:

In [4]:
def create_fx_forward_instrument_definition(dom_amount, fgn_amount, spot_rate, start_date, maturity_date):
    return models.FxForwardInstrument(
            dom_amount=dom_amount,
            fgn_amount=-fgn_amount,
            fgn_ccy="USD",
            ref_spot_rate=spot_rate,
            start_date=start_date.isoformat(),
            maturity_date=maturity_date,
            dom_ccy="GBP",
            instrument_type="FxForward")

gbpusd_fx_fwd_3y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['3Y'], spot_rate, trade_date, maturity_dates['3Y'])
gbpusd_fx_fwd_5y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['5Y'], spot_rate, trade_date, maturity_dates['5Y'])
gbpusd_fx_fwd_10y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['10Y'], spot_rate, trade_date, maturity_dates['10Y'])

## Supply Market Data

Before we can price our Fx forwards we need to ensure LUSID has the required FX and interest rates available. Let's upsert the
GBP/USD FX rates for the trade data and a few subsequent dates we'll use to price the FX Forwards. When providing market
data to LUSID we also need to identify the supplier of the data and the scope. The supplier is referenced further in the
notebook when we instruct LUSID how to source data for a particular aggregation.

### Upsert Spot Fx Rate

In [5]:
market_data_scope = 'fx-forward-pricing-nb-market-data'
market_supplier = 'Lusid'

trade_date_plus_one = datetime(2020, 6, 2, tzinfo=pytz.utc)
trade_date_plus_two = datetime(2020, 6, 3, tzinfo=pytz.utc)


def upsert_gbp_usd_fx_rate(rate, effective_at):
    upsert_quote_request = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="GBP/USD",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=rate,
            unit='rate'
        ),
        lineage='FxDataV    endorABC')

    # we need to insert the USD/GBP quote as well for use later in the notebook
    # when we're booking the sell leg of our forward as a transaction
    upsert_quote_request_inverse = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="USD/GBP",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=1/rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    response_gbp_usd = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"gbp-usd-01": upsert_quote_request})

    response_usd_gbp = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"usd-gbp-01": upsert_quote_request_inverse})

    if response_gbp_usd.failed or response_usd_gbp.failed:
        raise StopExecution(f"Failed to upload currency pairs:{response_gbp_usd.failed} or {response_usd_gbp.failed}")

    display(f"GBP/USD @ {rate} for {effective_at} uploaded to quote store.")
    display(f"USD/GBP @ {1/rate} for {effective_at} uploaded to quote store.")


upsert_gbp_usd_fx_rate(spot_rate, trade_date)
# drop in fx rate
upsert_gbp_usd_fx_rate(spot_rate - 0.0080, trade_date_plus_one)
# increase in fx rate
upsert_gbp_usd_fx_rate(spot_rate + 0.0080, trade_date_plus_two)


'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2428 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.804634695848085 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2588 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7944073721004131 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

### Interest Rate Curves

To price the FX Forwards we also need to supply the relevant interest rate curves for our domestic and foreign currencies. LUSID supports
storage of complex market data structures through the [Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr")
store. When upserting the interest rate curves we use a StructuredMarketDataId to not onl uniquely identify the curve but also provide
the supporting information required by the aggregation engine to correctly resolve the curves when we run our FX forward valuation.

In [6]:
def upsert_interest_rate_curve(ir_curve_json, scope, effective_at, market_asset):

    # provide the structured data file source and it's document format
    structured_market_data = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ir_curve_json)


    # create a unique identifier for our OIS yield curves
    structured_id = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="CurveVendorABC", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)

    upsert_request = models.UpsertStructuredMarketDataRequest(market_data_id=structured_id,
                                                          market_data=structured_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertStructuredMarketData
    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={market_asset : upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload interest rates curve {response.failed}")

    print(f"{market_asset} interest rate curve uploaded into scope={scope}")


def load_ois_curve_json(ccy):
    with open(f"data/{ccy}OIS50.json", "r") as ir_curve_json:
        return ir_curve_json.read()

ois_curve_gbp_json = load_ois_curve_json('GBP')
upsert_interest_rate_curve(ois_curve_gbp_json, market_data_scope, trade_date, "GBP/GBPOIS")

ois_curve_usd_json = load_ois_curve_json('USD')
upsert_interest_rate_curve(ois_curve_usd_json, market_data_scope, trade_date, "USD/USDOIS")

GBP/GBPOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data
USD/USDOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data


## Defining our Fx Forward Valuation

With our Fx forward instruments now defined we can move onto valuing them via LUSID's aggregation engine. But before we
can execute the aggregation we need to configure a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used")
instructing LUSID on how to value our Fx Forwards. Specifically we need to select the Fx Forward pricing model to use and identify
where the aggregation can resolve the market data the model needs.

### Selecting a Pricing Model

The pricing model we wish to use is passed in through defining a PricingContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters. For this notebook we'll use one of the supported Vendor models from VolMaster:

In [7]:
def create_pricing_context():
    vendor_model_rule = models.VendorModelRule(
        supplier="Lusid",
        model_name="Discounting",
        instrument_type="FxForward",
        parameters="{}")

    return models.PricingContext(
        model_rules=[vendor_model_rule]
    )

pricing_context = create_pricing_context()

### Selecting the Market Data

We can instruct LUSID on where to resolve market data required for pricing our Fx forwards through defining a MarketContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters. Recall that when we upserted our quote we passed in a supplier and scope. One of
the powerful features in LUSID is the ability to run valuations against market data from different suppliers, or even define market data retrieval rules
to fallback to suppliers should your primary supplier not have the required data.

However our example is trivial as we only require the spot rate we loaded in earlier. So we can define a simple market context :

In [8]:
def create_market_context():
    return models.MarketContext(
        # set rules for where we should resolve our rates data. In our case the interest rate curves.
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        # control default options for resolving market data. In our case simply default to the LUSID market_supplier
        # and market data scope we defined earlier.
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our FX Forward Recipe

Now that we've defined what pricing model to use and where to source the market data we can bring those instructions
together in a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used"):

In [9]:
def create_fx_forward_pricing_recipe():
    return models.ConfigurationRecipe(
            scope=scope,
            code="fx-forward-pricing-recipe",
            description="Price Fx Forwards using LUSID internal model",
            market=market_context,
            pricing=pricing_context
        )

fx_forward_pricing_recipe = create_fx_forward_pricing_recipe()

## Price the FX Forwards

Let's summarise our current state:
 * We've defined a set of GBP/USD FX Forwards for maturity in 3Y, 5Y and 10Y..
 * Loaded in the spot GBP/USD rates for the dates we're going to value our forwards against.
 * We've setup our Recipe configuring of how we would like to price our bond and where to source our required market data.

As we have no existing positions in the FX Forward booked against a portfolio we'll run our aggregation on an inlined portfolio.
That is a portfolio made up of a set of weighted instruments. In our case each request will be made up of only the specific
FX forward we're valuing.

### Run an Aggregation to Price our Forwards

In [10]:
def run_fx_forward_pricing_aggregation(maturity, effective_at, fx_forward_instrument_definition, fx_forward_pricing_recipe):
    # setup weighted instrument (only our gilt definition)
    fx_forward = models.WeightedInstrument(quantity=1, instrument=fx_forward_instrument_definition, holding_identifier=f"{maturity}-holding")

    # create our aggregation request made up of our recipe and the metrics we would like to calculate (Bond PV)
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at,
        inline_recipe=fx_forward_pricing_recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    # As we're running an inline aggregation we must wrap our original aggregation request with an inline aggregation
    # request and pass in our weighted instruments
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[fx_forward]
    )

    # https://www.lusid.com/docs/api#operation/GetAggregationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)


Start by running the 5Y forward valuation across the three dates we upserted quotes for. Recall the quotes were 1.2508 for
trade date, dropped by 0.0080 for day two and then increased by 0.0160 for day 3.

In [11]:
result_t = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_one, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_two, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 5Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 5Y Forward PV,01 Jun 2020,-0.006160
1,GBP/USD 5Y Forward PV,02 Jun 2020,-0.012605
2,GBP/USD 5Y Forward PV,03 Jun 2020,0.000202


Let's now take a look a look at the prices across the Forward curve for one date:

In [12]:
result_3y = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_5y = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_10y = run_fx_forward_pricing_aggregation('10Y', trade_date, gbpusd_fx_fwd_10y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_3y.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date, result_5y.data[0]['Holding/default/PV']],
    ['GBP/USD 10Y Forward PV', trade_date, result_10y.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.002024
1,GBP/USD 5Y Forward PV,01 Jun 2020,-0.006160
2,GBP/USD 10Y Forward PV,01 Jun 2020,-0.032075


## Pricing a Portfolio of FX Forwards

Up until now we've only priced a unit Fx Forward. Let's now take it a step further and build a simple of portfolio with
some FX forward positions. While we'll now be running our aggregation on an actual portfolio as opposed to just the Fx Forward instrument
we'll still make use of the same recipe we defined. This is because the rules regarding what model we'd like to use to price our
Fx Forward and where LUSID should retrieve market data from have not changed.


### Setting up our Portfolio

We'll firstly setup the portfolio that will hold our FX forward positions:

In [13]:
portfolio = "simple-fxfwd-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    succ, failed = format_portfolios_response(result)

    if not failed.empty:
        raise StopExecution(failed)

    return succ

create_portfolio(scope, portfolio, portfolio, "GBP")

,successful items
0,simple-fxfwd-portfolio-01


### Upserting our FX Forward Instrument

Previously as we were only pricing the FX Forward instrument we were only required to pass in it's definition. As we're now
booking transactions against the instrument we'll need to upsert the instrument into LUSID. This will provide us with an identifier
we can use to book against.

In [14]:
def upsert_fx_fwd_instrument(instrument_id, instrument_name, fx_fwd_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        # instrument display name
        name=instrument_name,
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        # our gilt instrument definition
        definition=fx_fwd_definition
    )}
    # Note we're using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition and not the base LUSID instruments.
    return instruments_api.upsert_lusid_instruments(bond_instrument_request)

gbpusd_fx_fwd_3y_instr_client_id = "gbp_usd_fwd_3y_12571"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_3y_instr_client_id, "GBP/USD 3Y @ 1.2571", gbpusd_fx_fwd_3y_instr_def)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gbpusd_fx_fwd_3y_instr_luid = instrument_creation_response.values[gbpusd_fx_fwd_3y_instr_client_id].lusid_instrument_id


### Setting up our FX Forward Transaction

Let's assume we're taking a position in the GBP/USD 3Y Fx Forward we setup. As the domestic and foreign notionals are
included in the instrument definition we simply need to setup a transaction type that gives use a holding in the instrument. However
there is no cash exchanged making a usual "Buy" transaction unsuitable so we'll have to create our own. For details
on movements take a look at the notebook [Generating holdings with the movements engine in LUSID](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/ibor/Generating%20holdings%20with%20the%20movements%20engine%20in%20LUSID.ipynb)


#### Creating a Transaction Type

For more information on transaction types have a read of [Configuring transaction types](https://support.finbourne.com/configuring-transaction-types) and try the
 TODO find notebook

We need a transaction that simply buys a unit of the forward with no cash movement that we'll call "FxFwdInstBuy"

In [15]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstBuy",
        description="Buy a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="LongLonger",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

#### Upserting our transaction

We can now ...

In [16]:
def upsert_fwd_transaction(scope, txn_id, txn_type, txn_ccy, instrument_id, trade_date, effective_at, portfolio, spot_rate, transaction_price, consideration):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type=txn_type,
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=effective_at.isoformat(),
        units=100000000,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency=txn_ccy),
        exchange_rate=1,
        transaction_currency=txn_ccy
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])

# our units are 100000000 as the PV in the forward pricing is calculated for a unit instrument. So to scale we'd need
# to upsert 100m units
units = 100000000
transaction_price = 1.2571
consideration = 100000000
# upsert the buy leg of the forward
upsert_fwd_transaction(scope, "fx_fwd_3y_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_3y_instr_client_id, trade_date, trade_date, portfolio, spot_rate, transaction_price, consideration)

### Value Portfolio

Mentione unitised valuatoin and PV from portfodlio

In [17]:
def run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
                                 op='Value'),
            models.AggregateSpec(key='Analytic/default/ValuationDate',
                                         op='Value'),
            models.AggregateSpec(key='Analytic/default/DomCcy',
                                 op='Value'),
            models.AggregateSpec(key='Analytic/default/FgnCcy',
                                 op='Value'),
            models.AggregateSpec(key='Analytic/default/StartDate',
                                 op='Value'),
            models.AggregateSpec(key='Analytic/default/MaturityDate',
                                 op='Value'),
            models.AggregateSpec(key='Portfolio/default/Name',
                                 op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
                                 op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
                                 op='Value'),
                        models.AggregateSpec("Holding/default/Error", "Value")
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)


result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,Portfolio PV,01 Jun 2020,-202392.125262
1,Portfolio PV,02 Jun 2020,-845476.464998
2,Portfolio PV,03 Jun 2020,432518.279429


In [18]:
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.002024
1,GBP/USD 3Y Forward PV,02 Jun 2020,-0.008455
2,GBP/USD 3Y Forward PV,03 Jun 2020,0.004325


## Forward Pricing Override with LUSID Static Model

Let's now assume that we've calculated priced our FX forwards externally to LUSID but would like the prices in LUSID to
value our portfolio. We can do so by changing the pricing model we use from "Discounting" model to a "SimpleStatic" model. For
the static model to retrieve the externally calculate Forward prices we need to firsty upsert those prices as market data quotes against the
FX forward insturment we defined earlier in the notebook

### Load FX Forward Prices into LUSID

In [30]:
def upsert_external_fx_forward_as_quote(fx_fwd_price, effective_at):
    spot_quote = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=gbpusd_fx_fwd_3y_instr_luid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                field='Mid'),
            effective_at=effective_at,
        ),
        metric_value=models.MetricValue(
            value=fx_fwd_price,
            unit='GBP'),
        lineage='InternalSystem')

    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=market_data_scope,
        request_body={"1": spot_quote})V

    if response.failed:
        raise StopExecution(f"Failed to upload FX Forward price:{response.failed}")

    print(f"FX Forward @{fx_fwd_price} uploaded into Quote store.")

upsert_external_fx_forward_as_quote(0.50000, trade_date)
upsert_external_fx_forward_as_quote(0.005000, trade_date_plus_one)
upsert_external_fx_forward_as_quote(0.005000, trade_date_plus_two)


FX Forward @0.5 uploaded into Quote store.
FX Forward @0.005 uploaded into Quote store.
FX Forward @0.005 uploaded into Quote store.


### Updating our Recipe to the Static Model

We now need to use a recipe that selects the Static model for pricing forwards. As we did earlier in the notebook we select
the model we'd like to use through the PricingContext of the recipe

In [20]:
def create_static_fx_forward_pricing_context():
    return models.PricingContext(
        # the default behaviour does not allow looking up data for pricing instruments so we must allow it.
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="FxForward",
                parameters="{}"
            )
        ]
    )

static_fx_forward_pricing_context = create_static_fx_forward_pricing_context()


### Instruct LUSID where to find the forward price

Additionally we need to make the LUSID aggregation aware of where to resolve the FX forward prices we've inserted.

In [26]:
def create_static_fx_forward_pricing_market_context():
    return models.MarketContext(
        market_rules=[
            # additional rule to resolve our quote
            models.MarketDataKeyRule(
                key='Fx.LusidInstrumentId.*',
                supplier=market_supplier,
                data_scope=market_data_scope,
                quote_type='Price',
                field='Mid'),
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope,
            manifest_level_of_detail="Full")
    )

    return market_context

static_fx_forward_pricing_market_context = create_static_fx_forward_pricing_market_context()

### Construct Static Model Fx Forward Recipe

Let's create a new recipe based on the pricing and market contexts we've just created

In [27]:
def create_static_fx_forward_pricing_recipe(scope, market_context, pricing_context):

    return models.ConfigurationRecipe(
        scope=scope,
        code="static-bond",
        description="Price bond using prices from the quote store.",
        market=market_context,
        pricing=pricing_context
    )

static_fx_forward_price_config_recipe = create_static_fx_forward_pricing_recipe(scope, static_fx_forward_pricing_market_context, static_fx_forward_pricing_context)


### Rerun Pricing with Static Model and compare with the Discounting Model

We can now price our Fx Fwd portfolio using the Static model (which uses our external FX forward price we fed in as a quote)
and compare the valuation with a corresponding aggregatoin but using the discounting model

In [31]:
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, static_fx_forward_price_config_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

#display(result_t.manifest)

,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.005037
1,GBP/USD 3Y Forward PV,02 Jun 2020,-0.011506
2,GBP/USD 3Y Forward PV,03 Jun 2020,0.001350


In [24]:
result_static_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date)
result_static_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_one)
result_static_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

result_discounting_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_discounting_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_discounting_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date, result_discounting_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_one, result_discounting_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_two, result_discounting_t_plus_two.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV (Discounting),01 Jun 2020,-202392.125262
1,GBP/USD 3Y Forward PV (Static),01 Jun 2020,-503677.646306
2,GBP/USD 3Y Forward PV (Discounting),02 Jun 2020,-845476.464998
3,GBP/USD 3Y Forward PV (Static),02 Jun 2020,-1150627.615063
4,GBP/USD 3Y Forward PV (Discounting),03 Jun 2020,432518.279429
5,GBP/USD 3Y Forward PV (Static),03 Jun 2020,135049.253257


### Update our Static Forward Rates To Match Discounting Model

Let's finally try to match the forward rates of the static and idiscounting model by using the discounting model to
calculate numbers and the feeding into quote store

In [25]:
# price the forward sintruments
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)

# upsert the forward prices of the instrument from discounting model into our quote store

upsert_external_fx_forward_as_quote(result_t.data[0]['Holding/default/PV'], trade_date)
upsert_external_fx_forward_as_quote(result_t_plus_one.data[0]['Holding/default/PV'], trade_date_plus_one)
upsert_external_fx_forward_as_quote(result_t_plus_two.data[0]['Holding/default/PV'], trade_date_plus_two)

# run pricing of our entire portfolios now using both static and discounting models

result_static_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date)
result_static_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_one)
result_static_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, static_fx_forward_price_config_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

result_discounting_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_discounting_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_discounting_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date, result_discounting_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date, result_static_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_one, result_discounting_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_one, result_static_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Discounting)', trade_date_plus_two, result_discounting_t_plus_two.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV (Static)', trade_date_plus_two, result_static_t_plus_two.data[0]['Holding/default/PV']]
])

FX Forward @-0.0020239212526166972 uploaded into Quote store.
FX Forward @-0.008454764649984195 uploaded into Quote store.
FX Forward @0.004325182794291728 uploaded into Quote store.


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV (Discounting),01 Jun 2020,-202392.125262
1,GBP/USD 3Y Forward PV (Static),01 Jun 2020,-503677.646306
2,GBP/USD 3Y Forward PV (Discounting),02 Jun 2020,-845476.464998
3,GBP/USD 3Y Forward PV (Static),02 Jun 2020,-1150627.615063
4,GBP/USD 3Y Forward PV (Discounting),03 Jun 2020,432518.279429
5,GBP/USD 3Y Forward PV (Static),03 Jun 2020,135049.253257
